# Exporting path data as InfoMap state files

[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/infomap.ipynb)  
[Download notebook](https://github.com/pathpy/pathpy/raw/master/doc/tutorial/infomap.ipynb)

In [ ]:
pip install git+git://github.com/pathpy/pathpy.git

In [ ]:
import pathpy as pp
from collections import Counter

In [ ]:
def pc_to_cnt(pc):
    cnt = Counter()
    for path in pc:
        p = tuple([v.uid for v in path.nodes])
        cnt[p] = pc[path]['frequency']
    return cnt

The following code generates a simple example instance for a set of paths. We use a custum attribute `frequency` to store the observed frequencies of each path. Right now, the construction of paths is based on an edge sequence. The interface is more cumbersome than it needs to be, and we are currently working on simplifying the API of the `Path` and `PathCollection` classes. There should be a better version by next week.

In [ ]:
pc = pp.PathCollection()
a = pp.Node('a')
b = pp.Node('b')
c = pp.Node('c')
d = pp.Node('d')

e1 = pp.Edge(a, b, uid='a-b')
e2 = pp.Edge(b, c, uid='b-c')
e3 = pp.Edge(c, d, uid='c-d')

# 15 observations of path a-b
# this path of length one should be ignored as there's no associated state node (empty memory)
pc.add(pp.Path(e1, frequency=15))

# 42 observations of path a-b-c
# this leads to state nodes a-b and b-c connected by a link with weight 42
pc.add(pp.Path(e1, e2, frequency=42))

# 41 observations of path a-b-c-d
# this leads to state nodes a-b-c and b-c-d connected by a link with weight 41
pc.add(pp.Path(e1, e2, e3, frequency=41))

In [ ]:
pp.io.infomap.to_state_file(pc_to_cnt(pc), 'test.state')
with open('test.state', 'r') as f:
    print(f.read())

In [ ]:
pc = pp.PathCollection()
a = pp.Node('a')
b = pp.Node('b')
c = pp.Node('c')
d = pp.Node('d')


# 1 observation of path a-b
e1 = pp.Edge(a, b, uid='a-b')
pc.add(pp.Path(e1, frequency=1))

# 1 observation of path b-c-d
e2 = pp.Edge(b, c, uid='b-c')
e3 = pp.Edge(c, d, uid='c-d')
pc.add(pp.Path(e2, e3, frequency=1))

# 1 observation of path c-d-a-c
e4 = pp.Edge(d, a, uid='d-a')
e5 = pp.Edge(a, c, uid='a-c')
pc.add(pp.Path(e3, e4, e5, frequency=1))

# 1 observation of path b-c-b-a-c
e6 = pp.Edge(c, b, uid='c-b')
e7 = pp.Edge(b, a, uid='b-a')
pc.add(pp.Path(e2, e6, e7, e5, frequency=1))

print(pc)

In [ ]:
pp.io.infomap.to_state_file(pc_to_cnt(pc), 'test.state', max_memory=1)
with open('test.state', 'r') as f:
    print(f.read())